In [3]:
# Preprocess data
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
encoded_labels = to_categorical(encoded_labels)

images = images / 255.0  # Normalize
images = images.reshape(images.shape[0], 100, 100, 1)  # Add channel dimension

X_train, X_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

Training set: 8 samples
Test set: 2 samples


In [4]:
# Build model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(100,100,1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_names), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

D:\Anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# Train model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10


D:\Anaconda\Lib\site-packages\keras\src\ops\nn.py:938: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
D:\Anaconda\Lib\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step - accuracy: 1.0000 - loss:

In [6]:
# Save model
model.save('./model/face_recognition_model.h5')

In [2]:
# Cell 1: Imports
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
# Cell 2: Load and preprocess data
dataset_path = './dataset/'
images = []
labels = []

for student in os.listdir(dataset_path):
    student_path = os.path.join(dataset_path, student)
    if os.path.isdir(student_path):
        for img_file in os.listdir(student_path):
            img_path = os.path.join(student_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img_resized = cv2.resize(img, (100, 100))
                images.append(img_resized)
                labels.append(student)

images = np.array(images)
labels = np.array(labels)

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
label_names = label_encoder.classes_

# Split data
X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)

# Normalize
X_train = X_train / 255.0
X_test = X_test / 255.0

# Reshape for CNN
X_train = X_train.reshape(-1, 100, 100, 1)
X_test = X_test.reshape(-1, 100, 100, 1)

# One-hot encode labels
y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(label_names))
y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(label_names))

print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")
print(f"Classes: {label_names}")

Training samples: 48
Testing samples: 12
Classes: ['Abdul Qayyum' 'Kashif' 'Salar Abdullah' 'Zain' 'shafqat' 'student1']


In [4]:
# Cell 3: Build model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(100,100,1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_names), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

D:\Anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# Cell 4: Train model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 991ms/step - accuracy: 0.1042 - loss: 2.2126 - val_accuracy: 0.4167 - val_loss: 1.8183
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 287ms/step - accuracy: 0.1667 - loss: 1.9277 - val_accuracy: 0.1667 - val_loss: 1.8857
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step - accuracy: 0.2083 - loss: 1.7973 - val_accuracy: 0.1667 - val_loss: 1.8010
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 423ms/step - accuracy: 0.2083 - loss: 1.7506 - val_accuracy: 0.2500 - val_loss: 1.7717
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 327ms/step - accuracy: 0.3125 - loss: 1.7580 - val_accuracy: 0.1667 - val_loss: 1.7735
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 298ms/step - accuracy: 0.3958 - loss: 1.7244 - val_accuracy: 0.1667 - val_loss: 1.7738
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 299ms/step - accuracy: 0.4167 - loss: 1.6915 - val_accuracy: 0.3333 - val_loss: 1.7360
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 325ms/step - accuracy: 0.2917 - loss: 1.6887 - val_accuracy: 0.4167 - val_loss:

In [6]:
# Cell 5: Save model
model.save('./model/face_recognition_model.h5')

In [2]:
# Imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import cv2

# --- 1. Load and Preprocess Data ---
def load_dataset(dataset_path):
    """Loads images and labels from the dataset folder."""
    images = []
    labels = []
    
    # Get student names from folder names
    label_names = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
    
    for label_name in label_names:
        student_dir = os.path.join(dataset_path, label_name)
        for img_file in os.listdir(student_dir):
            img_path = os.path.join(student_dir, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                images.append(img)
                labels.append(label_name)
                
    return np.array(images), np.array(labels), label_names

# Load the dataset
dataset_path = './dataset'
if not os.path.isdir(dataset_path):
    raise Exception(f"Dataset folder not found at '{dataset_path}'. Please create it and add student images.")

images, labels, label_names = load_dataset(dataset_path)
if len(images) == 0:
    raise Exception("Dataset is empty. Please run data_collection.py or add images manually.")

print(f"Loaded {len(images)} images for {len(label_names)} classes: {label_names}")

# Preprocess images
images_normalized = images / 255.0
images_reshaped = images_normalized.reshape(images.shape[0], 100, 100, 1)

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(
    images_reshaped, labels_categorical, test_size=0.2, random_state=42, stratify=labels_categorical
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# --- 2. Build and Compile CNN Model ---
num_classes = len(label_names)
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# --- 3. Train the Model ---
print("\nStarting model training...")
# Increased epochs to 20 for better learning with more data
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=5)

# --- 4. Evaluate the Model ---
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {accuracy * 100:.2f}%")

# --- 5. Save the Model and Labels ---
model_dir = './model'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save the trained model
model.save(os.path.join(model_dir, 'face_recognition_model.h5'))

# Save the label names for the attendance script to use
np.save(os.path.join(model_dir, 'label_names.npy'), label_names)

print(f"\nModel and label names saved successfully in the '{model_dir}' directory.")

Loaded 60 images for 2 classes: ['Salar Abdullah', 'Shafqat Rasool']
Training set: 48 samples
Test set: 12 samples


D:\Anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 98, 98, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 49, 49, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 47, 47, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 23, 23, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 33856)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       4,333,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,352,770 (16.60 MB)

 Trainable params: 4,352,770 (16.60 MB)

 Non-trainable params: 0 (0.00 B)


Starting model training...
Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 167ms/step - accuracy: 0.5208 - loss: 0.8248 - val_accuracy: 0.5000 - val_loss: 0.6693
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - accuracy: 0.5208 - loss: 0.6784 - val_accuracy: 0.5000 - val_loss: 0.6604
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 0.5208 - loss: 0.6497 - val_accuracy: 1.0000 - val_loss: 0.6139
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - accuracy: 0.6458 - loss: 0.6270 - val_accuracy: 0.6667 - val_loss: 0.5447
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - accuracy: 0.8542 - loss: 0.4294 - val_accuracy: 0.7500 - val_loss: 0.3910
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.8542 - loss: 0.3281 - val_accuracy: 1.0000 - val_loss: 0.2175
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - accuracy: 0.9792 - loss: 0.2023 - val_accuracy: 0.7500 - val_loss: 0.3072
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - accuracy: 1.0000 - loss


Test Accuracy: 100.00%

Model and label names saved successfully in the './model' directory.
